In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import json
import os
import re

In [ ]:
import matplotlib.font_manager as fm

font_files = fm.findSystemFonts(fontpaths=['/mnt/c/Windows/Fonts', '/mnt/c/Users/trommer/AppData/Local/Microsoft/Windows/Fonts'], fontext='otf')

for font_file in font_files:
    fm.fontManager.addfont(font_file)

import matplotlib as mpl
#mpl.rcParams['font.family'] = 'Neue Haas Grotesk Text Pro'
# mpl.rcParams['font.family'] = 'Linux Biolinum'
mpl.rcParams['font.family'] = 'CMU Sans Serif'

plt.rcParams['font.size'] = 8
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.axisbelow'] = True

colors = sns.color_palette('Set2')
sns.set_palette('Set2')

In [ ]:
def read_results(path):
    with open(path, 'r') as f:
        data = json.load(f)
    benchmarks = data['benchmarks']

    res = []
    for b in benchmarks:
        flat_dict = {**b['params'], **b['stats']}
        res.append(flat_dict)

    df = pd.DataFrame.from_records(res)
    b_name = os.path.basename(path)
    f_name, _ = os.path.splitext(b_name)
    df['filename'] = f_name
    return df

In [ ]:
ta = read_results('benchmark_results/221215_conv_layer_torchapprox.json')
tf = read_results('benchmark_results/221215_conv_layer_tfapprox.json')
df = pd.concat([ta, tf], ignore_index=True)
df.loc[df.bench_type.isna(), 'bench_type'] = 'tfapprox'
df.bench_type = df.bench_type.apply(lambda s: s[0] if isinstance(s, list) else s)
df.sort_values(['channels', 'bench_type'], inplace=True, ignore_index=True)
df = df[~(df.bench_type.str.contains('mul12') | df.bench_type.str.contains('mul16'))]
df = df[df.channels > 1].reset_index(drop=True)

In [ ]:
fig = plt.figure(figsize=(3.5,2.8))
ax = fig.add_subplot(111)


# ax.yaxis.set_tick_params(which='minor', size=2, width=0.75)

params = [
    
    ('mul8s', 'HTP, EvoApp. 8-Bit', 'o'),
    ('lut','LUT, TorchApprox', 's'),
    ('tfapprox', 'LUT, TFApprox', '^'),
]
for key, name, marker in params:
    models = df[df.bench_type.str.contains(key)]
    if key == 'mul8s':
        means = models.groupby('channels')['median'].agg('mean')
        stds = models.groupby('channels')['median'].agg('std')
        x = np.unique(models.channels)
        plt.errorbar(x, means.values * 1000, yerr=stds.values * 1000, label=name, capsize = 1.5, linestyle='--', ecolor=colors[0], elinewidth=0.8, markersize=4.0, marker=marker, markeredgecolor='black', barsabove=False, zorder=3)
        # plt.errorbar(x, means.values * 1000, yerr=stds.values * 1000)
    else:
        plt.plot(models.channels, models['median'] * 1000, label=name, marker=marker, linestyle='--', markersize=5.0, markeredgecolor='black', markeredgewidth=0.8)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Number of Channels', fontsize=7)
ax.set_ylabel('Median Execution Time [ms]', fontsize=7)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_position(('outward', 6))
ax.spines['bottom'].set_position(('outward', 4))
ax.spines['bottom'].set_bounds(2, 64)

ax.set_xticks(np.unique(df.channels))
ax.set_xticklabels(np.unique(df.channels), fontsize=7)
ax.xaxis.set_tick_params(which='major', size=2, width=1)
ax.xaxis.set_tick_params(which='minor', bottom=False)

ax.yaxis.set_tick_params(which='major', size=5, width=1, direction='out')
ax.yaxis.set_tick_params(which='minor', size=2, width=0.75, direction='out')
ax.yaxis.grid(linestyle='dotted')
#ax.set_ylim(2e-1, 1e3)
# ax.set_xlim(1.4,70)

ax.legend(loc='upper left', fontsize=7)
plt.tight_layout()
plt.savefig('/mnt/c/Users/trommer/Desktop/torchapprox-tcad/figures/plots/conv2d_benchmark.pdf', bbox_inches='tight')
plt.show()
    

In [ ]:
from scipy.stats import gmean
vals = df[df.bench_type == 'tfapprox']['median'] / df[df.bench_type.str.contains('mul8')].groupby('channels')['median'].agg('mean').values
vals

In [ ]:
pretty_names = {
    'alexnet' : 'AlexNet',
    'effcientnet_b0' : 'EfficientNetB0',
    'mobilenet_v2' : 'MobileNetV2',
    'resnet18' : 'ResNet18',
    'resnet50' : 'ResNet50',
    'vgg16' : 'VGG16',
}
df = read_results('benchmark_results/221205_networks.json')
df.loc[df.bench_type.isna(), 'bench_type'] = 'adaPT'
df.bench_type = df.bench_type.apply(lambda s: s[0] if isinstance(s, list) else s)
df = df.loc[~df.bench_type.str.contains("accurate")]
df.bench_architecture = df.bench_architecture.map(pretty_names)
df.sort_values(['bench_architecture', 'bench_type'], inplace=True, ignore_index=True)

In [ ]:
print(gmean(df[df.bench_type == 'adaPT']['median'] / df[df.bench_type.str.contains('lut')].groupby('bench_architecture')['median'].agg('mean').values))
print(gmean(df[df.bench_type == 'adaPT']['median'] / df[df.bench_type.str.contains('mul8s')].groupby('bench_architecture')['median'].agg('mean').values))
print(gmean(df[df.bench_type == 'lut']['median'] / df[df.bench_type.str.contains('mul8s')].groupby('bench_architecture')['median'].agg('mean').values))
print((df[df.bench_type == 'lut']['median'] / df[df.bench_type.str.contains('baseline')].groupby('bench_architecture')['median'].agg('mean').values))

In [ ]:
params = [
    # ('mul8s_1L2D', 'HTP Model 1L2D', '///'),
    ('mul8s', 'HTP, 8-Bit', '\\\\\\\\', 0),
    ('mul12s', 'HTP, 12-Bit', '////', 1),
    ('mul16s', 'HTP, 16-Bit', 'xxxx', 2),
    ('lut', 'LUT, TorchApprox', '/////', 1),
    ('adaPT', 'LUT, adaPT', '...', 2),
    ('baseline', 'Quant. only', None, 3),
]

width = 0.8/len(params)

fig = plt.figure(figsize=(7.1,2.8))
ax = fig.add_subplot(111)
colors = sns.color_palette('Set2')

for i, (key, label, hatch, c_idx) in enumerate(params):
    ans = df[df.bench_type.str.contains(key)]
    x = np.arange(len(np.unique(ans.bench_architecture))) + i*width
    if key == 'baseline':
        ax.bar(x, height=ans['median'].values * 1000, width=width, align='edge', edgecolor='grey', hatch='----', color = 'lightgrey', label=label, zorder=1)
    elif 'mul' in key:
        palette = sns.dark_palette(colors[0], 5, reverse=True)
        means = ans.groupby('bench_architecture')['median'].agg('mean')
        stds = ans.groupby('bench_architecture')['median'].agg('std')
        ax.bar(x, height=means.values * 1000, yerr=stds.values * 1000, capsize=1.2, error_kw = {'lw':0.6, 'mew': 0.6}, width=width, align='edge', edgecolor='black', color = palette[c_idx], label=label, zorder=2)
    else:
        ax.bar(x, height=ans['median'].values * 1000, width=width, align='edge', edgecolor='black', color = colors[c_idx], hatch=hatch, label=label, zorder=2)

labels = np.unique(df.bench_architecture)
ax.xaxis.set_ticks(np.arange(len(labels)) + len(params)/2 * width, labels, size=7)
ax.xaxis.set_tick_params(bottom=False)
# ax.set_xlim(-0.4, len(np.unique(df.bench_architecture))+0.2)

import matplotlib.ticker as ticker

ax.set_yscale('log')
ax.set_ylabel('Median Execution Time [ms]', fontsize=7)
# ax.set_ylim(5, 1e5)
ax.yaxis.set_major_locator(ticker.LogLocator(numticks=5))
ax.yaxis.set_tick_params(which='major', size=4, width=1, direction='out')
ax.yaxis.set_tick_params(which='minor', size=2, width=0.75, direction='out')
ax.yaxis.grid(which='major', linestyle='dotted')

ax.spines['left'].set_position(('outward', 8))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.legend(ncol=6, frameon=False, loc='upper left', bbox_to_anchor=(-0.02, 1.15), fontsize=7)

plt.tight_layout()
plt.savefig('/mnt/c/Users/trommer/Desktop/torchapprox-tcad/figures/plots/network_benchmark.pdf', bbox_inches='tight')
plt.show() 

In [ ]:
def report_accuracy(path, target_metric):
    df = pd.read_csv(path).loc[:,['Duration', 'Name', 'epochs',target_metric]]
    df = df.drop(df.index[df.Name.str.contains('QAT Model') | df.Name.str.contains('Baseline Model')])

    df.loc[df.Name.str.contains('Noise'),'type'] = 'noise'
    df.loc[df.Name.str.contains('Behavioral'),'type'] = 'behavioral'
    df.loc[df.Name.str.contains('Baseline'),'type'] = 'baseline'
    df.loc[df.Name.str.contains('HTP'),'type'] = 'htp'


    def time_to_float(s: str) -> float:
        t = float(re.findall(r'\d*.\d{1,2}', s)[0])
        if 'min' in s:
            t *= 60
        return t

    def mul_name(s: str) -> str:
        return re.findall(r'mul8s_[\w\d]{4}', s)[0]
        
    df.Duration = df.Duration.map(time_to_float)
    df['Multiplier'] = df.Name.map(mul_name)

    for m in np.unique(df.Multiplier):
        bl = df.loc[(df.Multiplier == m) & (df.type=='behavioral'), target_metric].values[0]
        df.loc[df.Multiplier == m, 'Deviation'] = df.loc[df.Multiplier == m, target_metric] - bl

        bl = df.loc[(df.Multiplier == m) & (df.type=='baseline'), 'Duration'].values[0]
        df.loc[df.Multiplier == m, 'Overhead'] = (df.loc[df.Multiplier == m, 'Duration'] / bl) -1.0
    df.Deviation = df.Deviation.abs()

    def print_result(df: pd.DataFrame, key: str, scale=1.0):
        mean = (df.groupby('type')[key].agg('mean') * scale).round(2)
        std = (df.groupby('type')[key].agg('std') * scale).round(2)

        print("\n" + key.upper())
        for m, s, l in zip(mean, std, mean.index):
            print(f"{l}: {m} $\pm$ {s}")
    
    
    print_result(df, target_metric, 100.0)
    print_result(df, 'Overhead', 100.0)
    print_result(df, 'Deviation', 100.0)

In [ ]:
lenet = pd.read_csv('accuracy_results/231013_affine_vs_per_tensor_lenet5.csv')
resnet = pd.read_csv('accuracy_results/231016_affine_vs_per_tensor_resnet8.csv')

In [ ]:

fig = plt.figure(figsize=(3.5, 3.0))
ax = fig.gca()

def plot_bitwidth_accuracy(ax, df):
    data = []
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_position(('outward', 8))
    ax.spines['right'].set_position(('outward', 8))
    ax.spines['left'].set_bounds(0.05,1.0)
    ax.spines['right'].set_bounds(0.05,1.0)


    ax.tick_params(axis='y', which='both', right=True, labelright=False, direction='in', labelsize=9)
    ax.tick_params(axis='x', labelsize=7)
    ax.set_ylabel('Top-1 Accuracy', fontsize=7)
    ax.yaxis.set_label_position('right')

    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0, decimals=0))
    ax.yaxis.grid(visible=True, ls=':')

    ax2 = ax.twiny()
    ax2.spines['top'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    ax2.spines['bottom'].set_position(('outward', 12))
    ax2.tick_params(axis='x', which='both', labelbottom=True, top=False, labeltop=False, labelsize=9)
    ax2.set_xticks(np.arange(3)*0.34+0.17, ["2-Bit", "3-Bit", "4-Bit"])

    for bw in [2,3,4]:
        for resolution in ["per_tensor", "per_channel"]:
            res = df[df.qtype == f"8x{bw}_affine_{resolution}"]
            data.append(res.test_acc_top1)

    pos = np.arange(len(data), dtype=np.float64)
    pos[1::2] -= 0.2
    bpl = ax.boxplot(data, positions=pos, widths=0.5, patch_artist=True, showfliers=False, medianprops={'color': 'black', 'alpha': 0.35}, capprops={'alpha': 0.35}, whiskerprops={'alpha': 0.35}, zorder=0)

    cols = sns.color_palette('Set2', 3)
    for i, (b, d, p) in enumerate(zip(bpl['boxes'], data, pos)):
        b.set_facecolor(cols[i//2])
        b.set(alpha=0.35)
        # x_idx = p * np.ones(len(d)) + 0.1 + np.random.rand(len(d)) * 0.25
        x_idx = p * np.ones(len(d)) + np.random.rand(len(d)) * 0.45 - 0.225
        ax.scatter(x_idx, d, color=cols[i//2], s=5, zorder=1, edgecolors='black', linewidth=0.5)

    ax.set_xticklabels(['tensor', 'channel'] * 3)

lenet = lenet[lenet.gradient_clip == 0.5]
plot_bitwidth_accuracy(ax, lenet)
plt.tight_layout()
plt.savefig('/mnt/c/Users/trommer/Desktop/torchapprox-tcad/figures/plots/quant_accuracy_lenet.pdf', bbox_inches='tight')
plt.show()


In [ ]:
set(resnet.qtype)

In [ ]:
fig = plt.figure(figsize=(3.5, 3.0))
ax = plt.gca()
baseline_acc = resnet[resnet.Name == 'Baseline Model'].test_acc_top1.iloc[0]
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.02))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.01))
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.02))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.01))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0, decimals=0))
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0, decimals=0))
ax.tick_params(axis='both', which='both', labelsize=9)
ax.set_ylabel('Top-1 Accuracy\nper-channel quantization', fontsize=7)
ax.set_xlabel('Top-1 Accuracy\nper-tensor quantization', fontsize=7)
ax.plot(np.linspace(0,baseline_acc,10), np.linspace(0,baseline_acc,10), zorder=0, color='grey', lw=0.5)
ax.grid(ls=":", lw=0.5)
ax.scatter(resnet[resnet.qtype == '8x4_affine_per_tensor'].test_acc_top1, resnet[resnet.qtype == '8x4_affine_per_channel'].test_acc_top1, color=colors[2], linewidth=0.5, edgecolors='black', zorder=1, alpha=0.75)
ax.hlines(baseline_acc, 0.8, 0.9, ls = ":", color='grey', lw=1.0, zorder=0)
ax.vlines(baseline_acc, 0.8, 0.9, ls = ":", color='grey', lw=1.0, zorder=0)
ax.text(0.802, baseline_acc-0.005, "FP32 Accuracy", color='grey')
ax.set_ylim(0.8, 0.89)
ax.set_xlim(0.8, 0.89)
plt.tight_layout()
plt.savefig('/mnt/c/Users/trommer/Desktop/torchapprox-tcad/figures/plots/quant_accuracy_resnet.pdf', bbox_inches='tight')
plt.show()

In [ ]:
def calc_metrics(path):
    df = pd.read_csv(path)
    df = df[~df.Name.str.contains("QAT") & ~df.Name.str.contains("Baseline Mode")]
    df.loc[:, 'mul_name'] = [n.split("-")[-1].strip() for n in df.Name]
    def name_to_method(name: str) -> str:
        if "Noise" in name:
            return "noise"
        return name.split("-")[1].strip().lower()
    df.loc[:, 'method'] = df.Name.apply(name_to_method)

    def time_to_float(s: str) -> float:
        t = float(re.findall(r'\d*.\d{1,2}', s)[0])
        if 'min' in s:
            t *= 60
        return t

    df["Duration"] = df["Duration"].map(time_to_float)

    for m in np.unique(df.mul_name):
        bl = df.loc[(df.mul_name == m) & (df.method=='behavioral'), 'test_loss'].values[0]
        df.loc[df.mul_name == m, 'Deviation'] = (df.loc[df.mul_name == m, 'test_loss'] - bl)

        bl = df.loc[(df.mul_name == m) & (df.method=='baseline'), 'Duration'].values[0]
        df.loc[df.mul_name == m, 'Overhead'] = (df.loc[df.mul_name == m, 'Duration']) 
    return df




In [ ]:
markers = ['o', 's', 'D', '^', 'X']
colors = sns.color_palette("Set2", 3)

def plot_experiment(testcase, ax, color, logmuls=False):
    df = calc_metrics(f"accuracy_results/231116_htp_comparison_{testcase}.csv")
    ans = []
    for method in np.unique(df.method):
        if logmuls:
            idx = ~df.mul_name.str.contains('mul8s')
        else:
            idx = df.mul_name.str.contains('mul8s')
        idx = idx & (df.method == method) 
        overhead = df[idx].Overhead.mean()
        deviation = df[idx].Deviation.abs().mean()
        ans.append((overhead, deviation))
    ans = np.array(ans)
    ans[:,1] /= np.max(ans[:,1])

    def is_pareto(p, points):
        p0 = points[:,0] < p[0]
        p1 = points[:,1] < p[1]
        return not np.any(p0 & p1)

    for p, m in zip(ans, markers): 
        if is_pareto(p, ans):
            alpha=1.0
        else:
            alpha=0.4
        ax.scatter(p[0], p[1], color=color, s=25, marker=m, edgecolor='black', zorder=2, alpha=alpha)
    ans = ans[ans[:,0].argsort()]
    pareto_idx = np.array([is_pareto(p, ans) for p in ans])
    plt.plot(ans[pareto_idx, 0], ans[pareto_idx,1], ms=0, lw=1.5, c=color, drawstyle='steps-post', zorder=1)
    return df

def plot_experiment_collection(ax, logmuls):
    plot_experiment('lenet', ax, colors[0], logmuls)
    plot_experiment('resnet', ax, colors[1], logmuls)
    plot_experiment('vgg', ax, colors[2], logmuls)
    ax.set_xscale('log')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    # ax.spines['bottom'].set_bounds(1.0, 5.0)
    ax.spines['bottom'].set_position(('outward', 8))
    ax.spines['left'].set_bounds(0.0, 1.0)
    ax.spines['left'].set_position(('outward', 8))
    ax.set_xlabel("Retraining Duration [s]", fontsize=7)
    ax.set_ylabel("Normalized Test Loss MAE", fontsize=7)
    ax.tick_params(axis='both', which='both', labelsize=8)
    # ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    # ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.25))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    # ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))
    ax.set_ylim(-0.04, 1.04)
    ax.set_xlim(10, 2000)
    ax.xaxis.set_minor_formatter(ticker.NullFormatter())
    ax.grid(lw=0.5, color='black', ls=':')
    if not logmuls:
        ax.set_title("EvoApprox 8-Bit", fontsize=9)
        h = [ax.plot([],[], color="lightgrey", mew=1.0, ms=5.0, mec='black', marker=m, ls="")[0] for m in markers]
        l = ax.legend(handles=h, labels=['Baseline', 'Behavioral', 'HTP', 'Linear', 'Noise'], title="Method", loc=(0.75, 0.60), fontsize=6, framealpha=1.0, labelspacing=0.7)
        ax.add_artist(l)
        h = [ax.plot([],[], color=c, marker='s', ls="")[0] for c in colors]
        ax.legend(handles=h, labels=['LeNet5', 'ResNet8', 'VGG16'], title="Experiment", loc=(0.75, 0.25), fontsize=6, framealpha=1.0, labelspacing=0.7)
    else:
        ax.set_title("Logarithmic Multipliers", fontsize=9)



fig = plt.figure(figsize=(7.1,3.0)) 
ax = plt.subplot(121)
plot_experiment_collection(ax, True)
ax = plt.subplot(122)
plot_experiment_collection(ax, False)
plt.tight_layout()
plt.savefig('/mnt/c/Users/trommer/Desktop/torchapprox-tcad/figures/plots/htp_model_performance.pdf', bbox_inches='tight')
    # ax.set_yscale('log')
plt.show()